## Chapter 15: Advanced C# for Performance

As your ASP.NET Core application grows, performance becomes critical. Every millisecond saved in request processing translates to better throughput and lower infrastructure costs. While the framework is highly optimized, the code you write has a significant impact. This chapter explores advanced C# features that help you write high‑performance, low‑allocation code. You’ll learn about `Span<T>` and `Memory<T>` for efficient memory handling, when to choose `StringBuilder` over string interpolation, the trade‑offs between `Task` and `ValueTask`, and how source generators can eliminate runtime overhead. By the end, you’ll have a toolkit of techniques to make your applications faster and more scalable.

### 15.1 Understanding Span<T> and Memory<T>

Modern .NET introduces `Span<T>` and `Memory<T>` to enable high‑performance, allocation‑free operations on contiguous memory regions. They are especially useful in scenarios like parsing, serialization, and working with buffers.

#### The Problem: Unnecessary Allocations

Consider parsing a substring from a larger string:

```csharp
string fullName = "John Doe";
string firstName = fullName.Substring(0, 4); // "John"
```

`Substring` allocates a new string on the heap. For many operations, these allocations add up, increasing garbage collection pressure.

#### Enter `Span<T>`

`Span<T>` is a **ref struct** that represents a contiguous region of memory. It can point to arrays, strings (as read‑only), unmanaged memory, or stack‑allocated memory. Crucially, operations on spans do not allocate heap memory.

```csharp
string fullName = "John Doe";
ReadOnlySpan<char> fullNameSpan = fullName.AsSpan();
ReadOnlySpan<char> firstNameSpan = fullNameSpan.Slice(0, 4); // No allocation!
Console.WriteLine(firstNameSpan.ToString()); // Outputs "John"
```

The `Slice` method simply creates a new view over the same memory. No new string is allocated until you explicitly call `ToString()`.

#### `Span<T>` vs. `Memory<T>`

- **`Span<T>`** is a `ref struct`, meaning it can only live on the stack. It cannot be stored in heap objects (e.g., fields of a class, async method state). Use `Span<T>` for synchronous, short‑lived operations.
- **`Memory<T>`** is a regular struct that can live on the heap. It can be stored in fields, used in async methods, etc. Use `Memory<T>` when you need to represent a buffer that may be used asynchronously.

You can get a `Span<T>` from a `Memory<T>` via the `.Span` property, but you must ensure the `Memory<T>` is pinned if used with unmanaged code.

#### Practical Examples

**1. Fast parsing of integers from a string**

```csharp
public static bool TryParseInt(ReadOnlySpan<char> span, out int result)
{
    // Custom parsing logic using span
    result = 0;
    bool negative = false;
    int index = 0;

    if (span.Length == 0) return false;

    if (span[0] == '-')
    {
        negative = true;
        index++;
    }

    for (; index < span.Length; index++)
    {
        char c = span[index];
        if (c < '0' || c > '9') return false;
        result = result * 10 + (c - '0');
    }

    if (negative) result = -result;
    return true;
}

// Usage
string input = "12345";
if (TryParseInt(input.AsSpan(), out int number))
    Console.WriteLine(number);
```

**2. Efficiently processing HTTP request paths**

In middleware, you might need to inspect the path without allocating strings:

```csharp
app.Use(async (context, next) =>
{
    var path = context.Request.Path.Value.AsSpan();
    if (path.StartsWith("/api/".AsSpan()))
    {
        // Handle API request
    }
    await next();
});
```

**3. Using `ArrayPool<T>` with spans**

For temporary buffers, use `ArrayPool<T>` to rent and return arrays, and wrap them in spans:

```csharp
byte[] buffer = ArrayPool<byte>.Shared.Rent(1024);
try
{
    Span<byte> span = buffer.AsSpan(0, 1024);
    // Fill span with data
}
finally
{
    ArrayPool<byte>.Shared.Return(buffer);
}
```

#### Performance Considerations

- Spans enable **zero‑copy** operations, reducing allocations and GC pressure.
- They are **fast** because they are just references and lengths; slicing is O(1).
- Use `ReadOnlySpan<T>` for read‑only access (e.g., from strings).

**When to use spans in ASP.NET Core:**

- Parsing query strings, headers, or JSON payloads.
- Custom formatters or serializers.
- Middleware that needs to examine request paths or bodies efficiently.

### 15.2 StringBuilders vs. String Interpolation (Performance Implications)

Strings in .NET are **immutable**. Every time you modify a string (e.g., via concatenation), a new string object is allocated. For simple operations, this is fine, but in loops or large string constructions, it becomes a performance bottleneck.

#### The Cost of Concatenation

```csharp
string result = "";
for (int i = 0; i < 1000; i++)
{
    result += i.ToString(); // Creates a new string each iteration!
}
```

This code creates thousands of intermediate strings, causing high memory usage and GC overhead.

#### Enter `StringBuilder`

`StringBuilder` maintains an internal buffer and appends to it without allocating a new string each time.

```csharp
var sb = new StringBuilder();
for (int i = 0; i < 1000; i++)
{
    sb.Append(i);
}
string result = sb.ToString(); // One final allocation
```

#### String Interpolation (C# 6+)

String interpolation is syntactically convenient:

```csharp
string name = "Alice";
int age = 30;
string message = $"Name: {name}, Age: {age}";
```

The compiler translates this into a call to `string.Format` (or, in recent versions, to a more efficient form using `DefaultInterpolatedStringHandler`). In .NET 6 and later, interpolated strings are optimized to use a **custom handler** that can reduce allocations.

For example, the above interpolation is compiled roughly to:

```csharp
var handler = new DefaultInterpolatedStringHandler(3, 2);
handler.AppendLiteral("Name: ");
handler.AppendFormatted(name);
handler.AppendLiteral(", Age: ");
handler.AppendFormatted(age);
string message = handler.ToStringAndClear();
```

This can be as efficient as a custom `StringBuilder` for many cases.

#### When to Use What?

- **Simple, single‑use interpolations** (like building a log message) are fine; the compiler’s handler avoids intermediate allocations.
- **Complex, multi‑step constructions** or **loops** should use `StringBuilder`.
- **Interpolation inside loops** can still be efficient if each iteration’s result is used immediately (no accumulation). However, if you’re accumulating, use `StringBuilder`.

**Example: Building a large CSV from a list**

```csharp
// Inefficient: using interpolation inside loop and concatenation
string csv = "";
foreach (var item in items)
{
    csv += $"{item.Id},{item.Name},{item.Price}\n"; // Allocates many strings
}

// Efficient: using StringBuilder
var sb = new StringBuilder();
foreach (var item in items)
{
    sb.Append(item.Id).Append(',').Append(item.Name).Append(',').Append(item.Price).Append('\n');
}
string csv = sb.ToString();
```

#### Benchmarking

Always measure. The difference can be dramatic. A simple benchmark (using BenchmarkDotNet) might show that `StringBuilder` is orders of magnitude faster for large constructions.

### 15.3 ValueTask vs. Task

Asynchronous programming is central to scalable web applications. The `Task` class represents an asynchronous operation. However, `Task` is a class, so every async method that returns `Task` allocates a new `Task` object on the heap—even if the method completes synchronously.

#### The Allocation Problem

Consider an async method that often completes synchronously (e.g., a cache hit):

```csharp
public async Task<string> GetCachedDataAsync(string key)
{
    if (_cache.TryGetValue(key, out string value))
        return value; // Synchronous completion, but still returns a Task

    value = await _slowSource.GetDataAsync(key);
    _cache[key] = value;
    return value;
}
```

Even when the cache returns immediately, this method allocates a `Task` (and likely an `AsyncStateMachine`). For high‑traffic endpoints, these allocations add up.

#### `ValueTask<T>` and `ValueTask`

`ValueTask<T>` (and non‑generic `ValueTask`) is a struct that can wrap either a result or a `Task`. If the operation completes synchronously, no `Task` is allocated; the result is stored directly in the struct. If it completes asynchronously, it falls back to using a `Task`.

```csharp
public ValueTask<string> GetCachedDataAsync(string key)
{
    if (_cache.TryGetValue(key, out string value))
        return new ValueTask<string>(value); // Synchronous, no Task allocation

    // Asynchronous path: use Task
    return new ValueTask<string>(_slowSource.GetDataAsync(key));
}
```

#### Guidelines for Using ValueTask

- **Use `ValueTask` when** your method often completes synchronously and you are in a high‑performance scenario.
- **Do not use `ValueTask`** if your method is always asynchronous; the slight overhead of `ValueTask` over `Task` in the async case may not be worth it.
- **Avoid** awaiting a `ValueTask` multiple times or concurrently (it can only be consumed once). If you need to store it, convert to `Task` using `.AsTask()`.
- **Be careful** with `ValueTask` in `async` methods that can complete synchronously—the compiler generates a state machine anyway, so the benefit is reduced. The real win is when you avoid `async` altogether for the sync path by returning a `ValueTask` from a non‑async method.

#### Example: Optimizing an In‑Memory Cache

```csharp
public class WeatherService
{
    private readonly ConcurrentDictionary<string, (string value, DateTime expiry)> _cache = new();
    private readonly IWeatherApi _api;

    public WeatherService(IWeatherApi api) => _api = api;

    public ValueTask<string> GetForecastAsync(string city)
    {
        if (_cache.TryGetValue(city, out var cached) && cached.expiry > DateTime.UtcNow)
            return new ValueTask<string>(cached.value);

        // Fall back to async API
        return new ValueTask<string>(FetchFromApiAsync(city));
    }

    private async Task<string> FetchFromApiAsync(string city)
    {
        var forecast = await _api.GetForecastAsync(city);
        _cache[city] = (forecast, DateTime.UtcNow.AddMinutes(5));
        return forecast;
    }
}
```

In this design, cache hits incur no Task allocation, reducing GC pressure.

### 15.4 Source Generators

**Source generators** are a Roslyn feature that allows you to inspect user code and generate additional C# source files during compilation. They enable scenarios like reducing reflection, which is often a performance bottleneck.

#### The Problem with Reflection

Reflection is powerful but slow. It involves runtime type inspection, method invocation overhead, and often allocations. For example, JSON serializers like `System.Text.Json` used reflection to read and write properties. In .NET 6+, `System.Text.Json` offers a source generator that creates optimized serialization code at compile time.

#### How Source Generators Work

A source generator runs during compilation, analyzing your code (e.g., types marked with attributes) and generating C# code that is compiled alongside your original code. The generated code is strongly typed and can be highly optimized.

#### Example: System.Text.Json Source Generator

Instead of using reflection‑based serialization:

```csharp
var options = new JsonSerializerOptions();
var product = new Product { Name = "Laptop", Price = 999.99m };
string json = JsonSerializer.Serialize(product, options); // Uses reflection by default
```

You can enable the source generator:

1. Create a partial `JsonSerializerContext` class:

```csharp
[JsonSerializable(typeof(Product))]
public partial class ProductJsonContext : JsonSerializerContext
{
}
```

2. Register it in your application:

```csharp
var options = new JsonSerializerOptions
{
    TypeInfoResolver = ProductJsonContext.Default
};
string json = JsonSerializer.Serialize(product, options); // Now uses generated code
```

The generated code directly accesses properties without reflection, improving performance and reducing memory allocations.

#### Creating a Simple Source Generator (Conceptual)

While writing a full source generator is beyond this chapter, let’s outline the concept. You create a class implementing `ISourceGenerator` and decorate it with `[Generator]`. In the `Execute` method, you use syntax trees to find relevant code and add source.

```csharp
[Generator]
public class HelloWorldGenerator : ISourceGenerator
{
    public void Execute(GeneratorExecutionContext context)
    {
        // Generate a class
        string source = @"
namespace Generated
{
    public static class HelloWorld
    {
        public static void SayHello() => System.Console.WriteLine(""Hello from generated code!"");
    }
}";
        context.AddSource("HelloWorld.g.cs", source);
    }

    public void Initialize(GeneratorInitializationContext context) { }
}
```

After adding this generator to a project, the `Generated.HelloWorld.SayHello()` method becomes available. For real generators, you’d analyze the compilation to generate tailored code.

#### Benefits in ASP.NET Core

- **JSON serialization**: Faster and allocation‑reduced API responses.
- **Dependency injection**: Some containers use source generators for faster service resolution.
- **Logging**: Source generators can create highly optimized log messages (e.g., `LoggerMessage` attribute in .NET 6+).

The `LoggerMessage` attribute, for instance, generates high‑performance logging methods that avoid boxing and parameter array allocations.

```csharp
private static readonly Action<ILogger, string, Exception?> _logHello =
    LoggerMessage.Define<string>(LogLevel.Information, new EventId(0), "Hello {Name}");

public void SayHello(ILogger logger, string name)
{
    _logHello(logger, name, null);
}
```

### Summary

In this chapter, you’ve explored advanced C# features that directly impact performance in ASP.NET Core applications:

- **`Span<T>` and `Memory<T>`** enable allocation‑free slicing and manipulation of memory, ideal for parsing and buffer handling.
- **`StringBuilder`** remains the go‑to for building large strings, though modern string interpolation is efficient for simple cases.
- **`ValueTask`** can reduce heap allocations in async methods that often complete synchronously.
- **Source generators** shift work from runtime to compile time, eliminating reflection overhead.

Applying these techniques thoughtfully will make your applications faster, more scalable, and better equipped to handle high load.

**Exercise:**

1. In a controller action, retrieve a large string (e.g., a long JSON payload). Use `Span<char>` to extract a substring without allocation and return it.
2. Write a method that builds a comma‑separated string from a list of 10,000 integers. Compare performance of `StringBuilder` vs. string interpolation inside a loop (use `Stopwatch` or BenchmarkDotNet).
3. Refactor a simple async cache method to return `ValueTask<T>` and measure allocations (use memory profiler or `GC.GetAllocatedBytesForCurrentThread`).
4. If you use `System.Text.Json` in your project, convert it to use the source generator and observe any performance improvements (optional, requires .NET 6+).

In the next chapter, **"Testing Your Application (Industry Standard),"** you’ll learn how to write unit tests, integration tests, and use mocking frameworks to ensure your application works correctly and remains reliable as it evolves.